# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

<a id="Этап_1"></a>
<a id="1.1"></a>

### Этап 1: Загрузим и изучим данные 
#### Подключение библиотек

[Вернуться к задачам исследования](#Задачи_исследования)

In [1]:
import pandas as pd                                            # Библиотека для работы с данными, предоставляет функции и структуры данных, такие как DataFrame и Series.
import numpy as np                                             # Фундаментальная библиотека для научных вычислений с поддержкой многомерных массивов и матриц.
import warnings                                                # Модуль для управления предупреждениями.
import random                                                  # Импорт модуля для генерации случайных чисел

import statsmodels.api as sm                                                # statsmodels.api используется для оценки статистических моделей
from statsmodels.stats.outliers_influence import variance_inflation_factor  # variance_inflation_factor из statsmodels используется для определения мультиколлинеарности

# Модули для работы с машинным обучением
from sklearn.model_selection import train_test_split           # Инструмент библиотеки sklearn для разделения исходных данных на обучающую, валидационную и тестовую выборку.

# Модели машинного обучения
from sklearn.linear_model    import LogisticRegression         # Модель логистической регрессии.
from sklearn.tree            import DecisionTreeClassifier     # Модель дерева решений.
from sklearn.ensemble        import RandomForestClassifier     # Модель случайного леса.
from sklearn.ensemble        import GradientBoostingClassifier # Модель градиентного бустинга.
from sklearn.dummy           import DummyClassifier            # Базовые модели для проверки на вменяемость.

# Инструменты для оценки моделей и метрики
from sklearn.model_selection import learning_curve             # Инструмент для построения кривых обучения.
from sklearn.model_selection import GridSearchCV               # Инструмент для выполнения поиска по сетке параметров модели.
from sklearn.metrics         import accuracy_score             # Метрика - точность классификации.
from sklearn.metrics         import confusion_matrix           # Метрика - матрица неточностей.
from sklearn.metrics         import roc_curve, roc_auc_score   # Метрики - ROC-кривая и ROC AUC Score.
from sklearn.metrics         import f1_score                   # Метрика - F1 Score.
from sklearn.preprocessing   import StandardScaler             # Инструмент для маштабирования данны.


%pip install seaborn
# Визуализация данных
import matplotlib.pyplot as plt                                # Библиотека для визуализации данных.
import seaborn as sns                                          # Библиотека для красивой визуализации статистических данных.

# Настрйоки отображения
# Устанавливаем формат отображения чисел с двумя знаками после запятой 
pd.options.display.float_format = '{:.2f}'.format

# Устанавливает фильтр предупреждений
warnings.filterwarnings("ignore")

random_state = random.seed(42)

Note: you may need to restart the kernel to use updated packages.


In [2]:
try:
    df_geo_0 = pd.read_csv('/Users/Edward/Code/DS+/csv/geo_data_0.csv', sep = ',')
    df_geo_1 = pd.read_csv('/Users/Edward/Code/DS+/csv/geo_data_1.csv', sep = ',')
    df_geo_2 = pd.read_csv('/Users/Edward/Code/DS+/csv/geo_data_2.csv', sep = ',')
except:
    df_geo_0 = pd.read_csv('/datasets/geo_data_0.csv', sep = ',')
    df_geo_1 = pd.read_csv('/datasets/geo_data_1.csv', sep = ',')
    df_geo_2 = pd.read_csv('/datasets/geo_data_2.csv', sep = ',')

In [3]:
print(df_geo_0.info())
print('-' * 35)
print(df_geo_0.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
-----------------------------------
      id    f0    f1   f2  product
0  txEyH  0.71 -0.50 1.22   105.28
1  2acmU  1.33 -0.34 4.37    73.04
2  409Wp  1.02  0.15 1.42    85.27
3  iJLyR -0.03  0.14 2.98   168.62
4  Xdl7t  1.99  0.16 4.75   154.04


In [4]:
print(df_geo_1.info())
print('-' * 35)
print(df_geo_1.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
-----------------------------------
      id     f0     f1    f2  product
0  kBEdx -15.00  -8.28 -0.01     3.18
1  62mP7  14.27  -3.48  1.00    26.95
2  vyE1P   6.26  -5.95  5.00   134.77
3  KcrkZ -13.08 -11.51  5.00   137.95
4  AHL4O  12.70  -8.15  5.00   134.77


In [5]:
print(df_geo_2.info())
print('-' * 35)
print(df_geo_2.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
-----------------------------------
      id    f0    f1    f2  product
0  fwXo0 -1.15  0.96 -0.83    27.76
1  WJtFt  0.26  0.27 -2.53    56.07
2  ovLUW  0.19  0.29 -5.59    62.87
3  q6cA6  2.24 -0.55  0.93   114.57
4  WPMUX -0.52  1.72  5.90   149.60


In [6]:
print('Колличество дубликатов в df_geo_0:', df_geo_0.duplicated().sum())
print('Колличество дубликатов в df_geo_1:', df_geo_1.duplicated().sum())
print('Колличество дубликатов в df_geo_2:', df_geo_2.duplicated().sum())

Колличество дубликатов в df_geo_0: 0
Колличество дубликатов в df_geo_1: 0
Колличество дубликатов в df_geo_2: 0


In [7]:
print('Колличество пропущенных значений в df_geo_0:')
print(df_geo_0.isnull().sum())
print('-' * 15)
print('Колличество пропущенных значений в df_geo_1:')
print(df_geo_1.isnull().sum())
print('-' * 15)
print('Колличество пропущенных значений в df_geo_2:')
print(df_geo_2.isnull().sum())
print('-' * 15)

Колличество пропущенных значений в df_geo_0:
id         0
f0         0
f1         0
f2         0
product    0
dtype: int64
---------------
Колличество пропущенных значений в df_geo_1:
id         0
f0         0
f1         0
f2         0
product    0
dtype: int64
---------------
Колличество пропущенных значений в df_geo_2:
id         0
f0         0
f1         0
f2         0
product    0
dtype: int64
---------------


In [8]:
# Список всех DataFrame, которые нужно объединить
dataframes = [df_geo_0, df_geo_1, df_geo_2]

# Используем функцию concat для объединения DataFrame
df = pd.concat(dataframes, ignore_index=True)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300000 entries, 0 to 299999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       300000 non-null  object 
 1   f0       300000 non-null  float64
 2   f1       300000 non-null  float64
 3   f2       300000 non-null  float64
 4   product  300000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 11.4+ MB


#### Вывод:

Анализ показал, что данные находятся в хорошем состоянии и подготовлены для проведения обучения.

#### Коэффициент Пирсона

Коэффициент корреляции Пирсона — это статистическая мера силы и направления линейной связи между двумя переменными. Коэффициент корреляции может принимать значения от -1 до +1. Значение +1 означает совершенную положительную линейную связь, значение -1 означает совершенную отрицательную линейную связь, а 0 означает отсутствие линейной связи.

Коэффициент корреляции Пирсона рассчитывается по формуле:

- $ r = \frac{n(\sum x_i y_i) - (\sum x_i)(\sum y_i)}{\sqrt{[n\sum x_i^2 - (\sum x)^2][n\sum y_i^2 - (\sum y_i)^2]}} $

где:

- $x$ и $y$ — значения переменных;

- $n$ — количество наблюдений.

Числитель:

- $ n(\sum x_i y_i) - (\sum x_i)(\sum y_i)$

Эта часть формулы представляет "ковариацию" между $x_i$ и $y_i$. Ковариация показывает, насколько переменные изменяются вместе. Если $x_i$ возрастает, когда $y_i$ возрастает, то ковариация будет положительной. Если $x_i$ уменьшается, когда $y_i$ возрастает, ковариация будет отрицательной. Если переменные не имеют явной связи, ковариация будет близка к нулю.

Знаменатель:

- $\sqrt{[n\sum x_i^2 - (\sum x)^2][n\sum y_i^2 - (\sum y_i)^2]}$

Эта часть формулы представляет собой произведение стандартных отклонений $x_i$ и $y_i$. Стандартное отклонение - это мера разброса значений переменной вокруг ее среднего значения. Знаменатель нормализует ковариацию, делая коэффициент корреляции Пирсона масштабно-инвариантным, т.е. он не зависит от масштаба измерения переменных.

Итоговый коэффициент корреляции:

Таким образом, коэффициент корреляции Пирсона представляет собой нормализованную ковариацию между двумя переменными. Это дает нам меру силы и направления линейной связи между переменными.

В контексте линейной регрессии коэффициент Пирсона может быть использован для оценки степени линейной зависимости между независимыми и зависимой переменными. Это может помочь в определении того, насколько переменные подходят для использования в модели линейной регрессии.

#### Линейная регрессия

Линейная регрессия — это статистический метод, используемый для моделирования и анализа взаимосвязей между переменными, где одна или несколько независимых переменных используются для прогнозирования значения зависимой переменной.

Модель линейной регрессии предполагает линейную зависимость между независимыми и зависимыми переменными, и она строится по следующей формуле:

- $ Y=β_0 + β_1 X_1 + β_2 X_2 +…+ β_n X_n + ϵ $

где:

- $Y$ — зависимая переменная, значение которой мы хотим предсказать;

- $X_1, X_2 ,…, X_n$ — независимые переменные, которые используются для предсказания значения Y;

- $β_0$ — это константа, которая представляет собой точку пересечения линии регрессии с осью Y, когда все независимые переменные равны нулю;

- $β_1, β_2 ,…, β_n$ — это коэффициенты регрессии, которые представляют изменение зависимой переменной $Y$ на единицу, при изменении соответствующей независимой переменной $X$, при условии, что все остальные независимые переменные остаются неизменными;

- $ϵ$ — это ошибка, которая представляет разницу между фактическим значением зависимой переменной и значением, предсказанным моделью линейной регрессии.

#### Оценка параметров

Для оценки параметров $β_0, β_1 ,…, β_n$ обычно используется метод наименьших квадратов (OLS). Этот метод минимизирует сумму квадратов ошибок (SSE), которые представляют собой разницу между фактическими и предсказанными значениями зависимой переменной.

- $ [ SSE = \sum_{i=1}^{n} (Y_i - \hat{Y}_i)^2] $

где:

- $Y_i$ — фактическое значение зависимой переменной для $i$-го наблюдения;

- $\hat{Y}_i$— значение зависимой переменной, предсказанное моделью линейной регрессии для $i$-го наблюдения.

#### Подбор значений наименьших квадратов

Метод наименьших квадратов (OLS) минимизирует сумму квадратов ошибок (SSE), находя значения параметров $β_0, β_1, …, β_n$, которые минимизируют SSE. Для этого, необходимо взять производные SSE по каждому параметру, приравнять их к нулю и решить полученные уравнения. Это приведет к аналитическому решению, описанному ниже.

#### Аналитическое решение

Линейная регрессия может быть представлена через аналитическое решение. Аналитическое решение можно получить, минимизируя сумму квадратов ошибок (SSE), что приводит к системе линейных уравнений, известной как уравнения нормального состояния. Для простой линейной регрессии с одной независимой переменной аналитическое решение может быть найдено из следующих формул:

- $ β_1 = \frac{n(\sum xy) - (\sum x)(\sum y)}{n\sum x^2 - (\sum x)^2} $

- $ β_0 = \frac{\sum y - β_1\sum x}{n} $

#### Предпосылки линейной регрессии

Для того чтобы модель линейной регрессии была действительной, должны выполняться следующие предпосылки:

- Линейность: Зависимость между зависимой и независимыми переменными должна быть линейной;

- Нормальность ошибок: Ошибки должны быть нормально распределены;

- Гомоскедастичность ошибок: Дисперсия ошибок должна быть одинаковой для всех значений независимых переменных;

- Независимость ошибок: Ошибки должны быть независимыми друг от друга;

- Отсутствие мультиколлинеарности: Независимые переменные не должны быть сильно коррелированными друг с другом.

#### Почему не стоит использовать интерполиационный полином Лагранжа

Интерполиационный полином Лагранжа используется для нахождения полинома, который проходит через заданный набор точек. Однако, этот метод не подходит для задач линейной регрессии по нескольким причинам:

- Переобучение: Интерполиационный полином Лагранжа будет проходить точно через каждую точку данных, что может привести к переобучению модели. Модель будет идеально описывать обучающий набор данных, но может плохо работать на новых данных;

- Высокая степень полинома: Интерполиационный полином Лагранжа может иметь высокую степень, что делает его сложным и неустойчивым. Например, если у нас есть 10 точек данных, интерполиационный полином будет 9-й степени;

- Вычислительная сложность: Расчет коэффициентов интерполиационного полинома Лагранжа может быть вычислительно сложным для больших наборов данных.

Из-за этих причин, интерполиационный полином Лагранжа не рекомендуется для задач линейной регрессии. Вместо этого, лучше использовать метод наименьших квадратов для нахождения параметров линейной регрессии.

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [ ]  Весь код выполняется без ошибок
- [ ]  Ячейки с кодом расположены в порядке исполнения
- [ ]  Выполнен шаг 1: данные подготовлены
- [ ]  Выполнен шаг 2: модели обучены и проверены
    - [ ]  Данные корректно разбиты на обучающую и валидационную выборки
    - [ ]  Модели обучены, предсказания сделаны
    - [ ]  Предсказания и правильные ответы на валидационной выборке сохранены
    - [ ]  На экране напечатаны результаты
    - [ ]  Сделаны выводы
- [ ]  Выполнен шаг 3: проведена подготовка к расчёту прибыли
    - [ ]  Для всех ключевых значений созданы константы Python
    - [ ]  Посчитано минимальное среднее количество продукта в месторождениях региона, достаточное для разработки
    - [ ]  По предыдущему пункту сделаны выводы
    - [ ]  Написана функция расчёта прибыли
- [ ]  Выполнен шаг 4: посчитаны риски и прибыль
    - [ ]  Проведена процедура *Bootstrap*
    - [ ]  Все параметры бутстрепа соответствуют условию
    - [ ]  Найдены все нужные величины
    - [ ]  Предложен регион для разработки месторождения
    - [ ]  Выбор региона обоснован